# Домашнее задание на тему "Морфология", часть 1

## Короткое введение

Для начала встроим проверку PEP-8, чтобы код был красивым:

In [13]:
%load_ext pycodestyle_magic
%pycodestyle_on

Импортируем несколько важных библиотек, которые нам понадобятся на протяжении всей работы.

In [3]:
from pprint import pprint
import json
import collections as c
import time

## Задание 1

Импортируем Mystem:

In [8]:
from pymystem3 import Mystem

И облегчим себе жизнь:

In [9]:
m = Mystem()

Откроем текст, распарсим его при помощи Mystem и посчитаем время.

In [10]:
with open('Luzhin.txt', encoding='utf-8') as f:
    book = f.read()
%time ana = m.analyze(book)

CPU times: user 789 ms, sys: 186 ms, total: 976 ms
Wall time: 7.48 s


Создадим новый файл, где будут лежать jsonы. Чтобы файл выглядел красиво, как набор твитов из предыдущей домашки, разделим их на строчки.

In [14]:
with open('Luzhin_mystem.json', 'w', encoding='utf-8') as f:
    for word in ana:
        if 'analysis' in word:
            json.dump(word, f, ensure_ascii=False)
            f.write('\n')

Ура!

## Задание 2

Импортируем nltk и pymorphy2. Из них возьмем нужные нам модули.

In [1]:
import nltk
from nltk.tokenize import word_tokenize
from pymorphy2 import MorphAnalyzer
morph = MorphAnalyzer()

Токенизируем текст при помощи nltk и замеряем время.

In [11]:
%time tokens = word_tokenize(book)

CPU times: user 845 ms, sys: 15.3 ms, total: 860 ms
Wall time: 919 ms


К сожалению, на этом моменте началось что-то странное: %time влиял на дальнейшую работу файла и в том числе мешал работать функции print(). Поэтому замерил время я другим образом.

Итак, парсим каждое слово, добавляем его в новый список и замеряем время работы.

In [12]:
words_parsed = []
start = time.time()
for token in tokens:
    par = morph.parse(token)
    words_parsed.append(par)
end = time.time()
print(end - start)

25.936647176742554


Создаем словари, в котором есть само слово и список вариантов и их свойств. Добавляем все эти словари в один список:

In [15]:
allthemwords = []
for word in words_parsed:
    word_dict = {}
    for var in word:
        if var == word[0]:
            word_dict['word'] = var.word
            word_dict['vars'] = []
        var_dict = {}
        var_dict['tag'] = str(var.tag).split(',')
        var_dict['normal_form'] = var.normal_form
        var_dict['score'] = var.score
        word_dict['vars'].append(var_dict)
    allthemwords.append(word_dict)

Создаем новый файл, куда дампаем все эти словари.

In [16]:
with open('Luzhin_pymorphy.json', 'w', encoding='utf-8') as f:
    for word in allthemwords:
        json.dump(word, f, ensure_ascii=False)
        f.write('\n')

## Задание 3

Чтобы не повторяться, создадим две функции.

Первая берет текст, проанализированный при помощи Mystem, и если часть речи совпадает с заданной, то единица прибавляется к числу count. К числу all_count единица прибавляется каждый раз, когда "слово" не пустое, т.е. там есть ключ 'analysis'. В итоге мы получаем долю слов, которую составляет заданная часть речи.

In [17]:
def ratio(s):
    all_count = 0
    count = 0
    for word in ana:
        if 'analysis' in word and word['analysis']:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            if pos == s:
                count += 1
            all_count += 1
    return (count / all_count)

Вторая функция составляет топ-n список самых популярных слов определенной части речи. Мы выуживаем название части речи из закопанного ключа gr и, если часть речи совпадает с заданной, добавляем в список лексему. Из списка потом мы составляем счетчик и печатаем n самых популярных лексем и к каждой лексеме -- отношение ко всем словам в тексте.

In [18]:
def top(num, s):
    all_count = 0
    s_list = []
    for word in ana:
        if 'analysis' in word and len(word['analysis']) > 0:
            gr = word['analysis'][0]['gr']
            pos = gr.split('=')[0].split(',')[0]
            if pos == s:
                s_list.append(word['analysis'][0]['lex'])
            all_count += 1
    s_count = c.Counter(s_list)
    i = 1
    for s in s_count.most_common(num):
        print(str(i) + '.', s[0] + ':', s[1] / all_count)
        i += 1

Теперь сначала создаем список из всех частей речи, которые есть в тексте, и потом -- для каждой из них печатаем получившееся отношение.

In [19]:
all_pos = []
for word in ana:
    if 'analysis' in word and len(word['analysis']) > 0:
        gr = word['analysis'][0]['gr']
        pos = gr.split('=')[0].split(',')[0]
        if pos not in all_pos:
            all_pos.append(pos)
for pos in all_pos:
    print(pos + ':', ratio(pos))

ADV: 0.07462339774619525
SPRO: 0.07981257018936606
V: 0.1999767648995082
CONJ: 0.08418851411532355
PR: 0.10480966580180459
S: 0.25426944971537
APRO: 0.04465011811176083
A: 0.09013282732447818
PART: 0.03729233628935445
ADVPRO: 0.021821631878557873
INTJ: 0.0008906788521860357
NUM: 0.005111722108198118
ANUM: 0.002420322967896836


Топ-списки теперь сделать довольно легко при помощи нашей функции:

In [20]:
print('Топ-20 глаголов:')
top(20, 'V')
print()
print('Топ-20 наречий:')
top(20, 'ADV')

Топ-20 глаголов:
1. быть: 0.016651822019130232
2. сказать: 0.006699453975138442
3. становиться: 0.0028850249777330286
4. говорить: 0.0024590481353831857
5. мочь: 0.0023428726329241373
6. знать: 0.0018588080393447703
7. сидеть: 0.0014909189482244511
8. находить: 0.0014909189482244511
9. играть: 0.0014134686132517523
10. спрашивать: 0.0013941060295085776
11. выходить: 0.001374743445765403
12. глядеть: 0.0012972931107927041
13. продолжать: 0.0012779305270495295
14. думать: 0.0012004801920768306
15. стоять: 0.0011423924408473067
16. смотреть: 0.0011423924408473067
17. понимать: 0.0010843046896177825
18. увидеть: 0.0010649421058746078
19. оказываться: 0.0010455795221314332
20. стараться: 0.0010262169383882585

Топ-20 наречий:
1. еще: 0.0028850249777330286
2. очень: 0.0024590481353831857
3. уже: 0.0021298842117492157
4. опять: 0.001936258374317469
5. вдруг: 0.0018975332068311196
6. теперь: 0.0016458196181698486
7. сразу: 0.0012779305270495295
8. хорошо: 0.0010843046896177825
9. быстро: 0.001

## Задание 4

Создаем список токенов без ненужных знаков препинания и приводим к нижнему регистру. Создаем список всех биграмм, а из них -- счетчик. Дальше пользуемся функцией most_common.

In [21]:
tokens = [w.lower() for w in word_tokenize(book) if w.isalpha()]
bgrm = nltk.bigrams(tokens)
bgrms = list(map(' '.join, bgrm))
bgrms_counter = c.Counter(bgrms)
for answer in bgrms_counter.most_common(25):
    print(answer[0] + ':', answer[1])

и в: 73
сказал лужин: 58
что он: 53
лужин и: 49
и не: 45
у него: 45
и он: 42
и с: 42
как будто: 41
сказал он: 40
и все: 38
и лужин: 36
и вдруг: 35
все это: 34
он не: 34
сказала она: 34
то что: 32
глядя на: 30
так как: 29
с ним: 28
о том: 28
не было: 28
а потом: 28
и что: 27
и на: 26


То, что самая популярная биграмма: "и в" -- неудивительно. Оба слова -- стоп-слова, оба очень частотные. Но интересно, что "сказал лужин" состоит не из стоп-слов. Тем временем ни жена, ни мать жены, ни Валентинов не встречаются в этих биграммах. Возможно, Лужин говорит чаще остальных в романе. Но может быть Набоков просто любит упоминать его имя: слово "лужин" встречается в еще нескольких биграммах этого списка.

Практически такая же история (если говорить о коде) с триграммами:

In [22]:
trgrm = nltk.trigrams(tokens)
trgrms = list(map(' '.join, trgrm))
trgrms_counter = c.Counter(trgrms)
for answer in trgrms_counter.most_common(25):
    print(answer[0] + ':', answer[1])

сказал лужин и: 24
о том что: 16
на следующий день: 10
сказал он и: 9
сказала она и: 8
с тех пор: 7
в первый раз: 6
на то что: 6
о том как: 6
что все это: 6
ему показалось что: 6
в том что: 5
сказал лужин старший: 5
как только он: 5
тех пор как: 5
в это мгновение: 5
ни с того: 5
с того ни: 5
того ни с: 5
ни с сего: 5
у себя в: 5
с этого дня: 5
после того как: 5
не зная что: 5
у него было: 5


Тут Лужин вообще встает на первое место. К тому же, 5 раз встречается "сказал Лужин-старший" (отец Лужина), что несомненно повлияло на частотность биграммы "сказал Лужин". Остальные триграммы особо ничем не выделяются: многие из них входят в список топ-100 самых частотных триграмм в НКРЯ.

*Миша Сонькин*